# Compute statistical informations: mean, std and percentiles : 

This notebook computes the mean, std and needed percentiles of waiting time for the arrival time of each train_type, hour and station. It uses the historical data of the SBB. It will write the created data as an ORC file in the home of the user running the notebook.

Note, this notebook takes approximately 30 minutes to run.  

#### Set up spark:

In [1]:
%%configure
{"conf": {
    "spark.app.name": "dslab-group_final"
}}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8983,application_1589299642358_3520,pyspark,idle,Link,Link,
8996,application_1589299642358_3534,pyspark,idle,Link,Link,
9028,application_1589299642358_3573,pyspark,idle,Link,Link,
9035,application_1589299642358_3582,pyspark,idle,Link,Link,
9054,application_1589299642358_3599,pyspark,idle,Link,Link,
9055,application_1589299642358_3601,pyspark,idle,Link,Link,
9059,application_1589299642358_3605,pyspark,idle,Link,Link,
9065,application_1589299642358_3611,pyspark,idle,Link,Link,
9067,application_1589299642358_3614,pyspark,idle,Link,Link,
9070,application_1589299642358_3618,pyspark,idle,Link,Link,


#### Imports:

In [2]:
import networkx as nx
from geopy.distance import distance as geo_distance
from pyspark.sql import Row
import pyspark.sql.functions as f
from pyspark.sql.functions import *
from pyspark.sql.types import FloatType
from networkx.algorithms.shortest_paths.weighted import dijkstra_path
from scipy.stats import norm

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
9096,application_1589299642358_3657,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
%%local
import os
username = os.environ['JUPYTERHUB_USER']

In [4]:
%%send_to_spark -i username -t str -n username

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Successfully passed 'username' as 'username' to Spark kernel

In [5]:
# Load stops in Zürich area
stops_zurich = spark.read.format('orc').load("/user/gottraux/nodes.orc".format(username))\
                                        .select('stop_id').distinct().rdd.flatMap(lambda x:x).collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Taken from find_train_type_correspondace.ipynb, to transform `verkehrsmittel_text` in the same format
replace_actual = {
    'BUS': 'Bus', # Buses
    'B': 'Bus',
    'NFB': 'Bus',
    'KB': 'Bus',
    'BAT': 'Bus',
    'Trm': 'Tram', # Trams
    'T': 'Tram',
    'TRAM': 'Tram',
    'ATZ': 'ARZ', #AutoZug
    'D': 'RE', # Regional
    'RB': 'R',
    'M': 'Metro', # Metro
    'ICE': 'IC', # InterCityExpress, but routes.txt doesn't have that category
    'IRE': 'IR', # InterRegioExpress, but routes.txt doesn't have that category
    'BN': '', # Night
    'TN': '',
    'SN': '',
    'BT': '',
    'VAE': '', # Panorama trains in the Alps
    'PE': '',
    'TER': '', # France
    'TE2': '',
    'RJX': '', # International
    'null': '', # Other
    '': ''
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
@udf("string")
def replace_verkehrsmittel_text(text):
    if text in replace_actual.keys():
        return replace_actual[text]
    else:
        return text

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
actual = spark.read.format('orc').load('/data/sbb/orc/istdaten/')\
                                 .where((col("ankunftszeit") != "") & (col("an_prognose") != ""))\
                                 .select(col('bpuic').alias('stop_id'), # Transform bpuic
                                         replace_verkehrsmittel_text(col('verkehrsmittel_text')).alias('verkehrsmittel_text'), # Translate `verkehrsmittel_text`
                                         from_unixtime(unix_timestamp('ankunftszeit', 'dd.MM.yyy HH:mm')).alias('ankunftszeit'),
                                         from_unixtime(unix_timestamp('an_prognose', 'dd.MM.yyy HH:mm:ss')).alias('an_prognose'),
                                         col('an_prognose_status'))\
                                 .where(col('stop_id').isin(stops_zurich))\
                                 .where(col("verkehrsmittel_text") != "")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
actual.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+-------------------+-------------------+------------------+
|stop_id|verkehrsmittel_text|       ankunftszeit|        an_prognose|an_prognose_status|
+-------+-------------------+-------------------+-------------------+------------------+
|8503000|                 EC|2018-09-03 21:51:00|2018-09-03 21:53:40|              REAL|
|8503000|                 EC|2018-09-03 10:51:00|2018-09-03 10:51:28|              REAL|
|8503000|                 IC|2018-09-03 07:00:00|2018-09-03 07:00:01|              REAL|
|8503000|                 IC|2018-09-03 21:23:00|2018-09-03 21:24:55|              REAL|
|8503000|                 IC|2018-09-03 08:26:00|2018-09-03 08:28:06|              REAL|
|8503000|                 IC|2018-09-03 17:53:00|2018-09-03 17:55:21|              REAL|
|8503000|                 IC|2018-09-03 10:00:00|2018-09-03 09:59:07|              REAL|
|8503000|                 EC|2018-09-03 12:51:00|2018-09-03 12:52:50|              REAL|
|8503000|            

In [10]:
actual = (actual.withColumn('hour', hour(col('ankunftszeit')))
               .withColumn('diff', unix_timestamp('an_prognose') - unix_timestamp('ankunftszeit')) # Compute delay
               .select(col('stop_id'), col('verkehrsmittel_text'), col('an_prognose_status'), col('hour'), col('diff')))

# Take only trains at reasonable hours
actual = actual.where((col('hour') >= 8) & (col('hour') <= 20))

# Count how many 'REAL' an_prognose status there are, if there are >= 10 we say that there are enough of them
enough_real_values = actual.where(col('an_prognose_status') == "REAL")\
                        .groupBy('stop_id', 'verkehrsmittel_text', 'hour')\
                        .agg(count('diff').alias('count'))\
                        .withColumn("enough_values", col("count") >= 10)\
                        .select(col('stop_id').alias('stop_id2'), 
                                col('verkehrsmittel_text').alias('verkehrsmittel_text2'), 
                                col('hour').alias('hour2'), col('enough_values'))

# Left join with enough_values
actual = actual.join(enough_real_values, (actual.stop_id == enough_real_values.stop_id2) &\
                                        (actual.verkehrsmittel_text == enough_real_values.verkehrsmittel_text2) &\
                                        (actual.hour == enough_real_values.hour2), "left")\
                    .select('stop_id', 'verkehrsmittel_text', 'an_prognose_status', 'hour', 'diff', 'enough_values')

# Set `enough_values` to False for trains that didn't have any 'REAL' `an_prognose_status`
actual = actual.na.fill(False)

# Take only REAL an_prognose_status if there are enough values
# If there aren't enough we use all the data available
actual = actual.where((~(col("enough_values"))) | (col("an_prognose_status") == "REAL"))

# Compute mean and std
actual = actual.groupBy('stop_id', 'verkehrsmittel_text', 'hour')\
                .agg(mean("diff").alias("mean"), stddev("diff").alias("std"), count("diff").alias('number_of_records'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
#actual = actual.where(col('number_of_records') >= 5).select('stop_id', 'verkehrsmittel_text', 'hour', 'mean', 'std')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
@udf("float")
def compute_delay_percentile(mean, std, percentile):
    X = norm(loc=mean, scale=std)
    return float(X.ppf(percentile))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# We now compute and add all the percentiles 90-99
# That means, given the delay distribution how much time do we have to wait to be 'percentile'% sure we'll be arrived by then
# This will be useful when validating paths to only take safe edges
actual = actual.withColumn('p_90', compute_delay_percentile(col('mean'), col('std'), lit(0.9)))\
                .withColumn('p_91', compute_delay_percentile(col('mean'), col('std'), lit(0.91)))\
                .withColumn('p_92', compute_delay_percentile(col('mean'), col('std'), lit(0.92)))\
                .withColumn('p_93', compute_delay_percentile(col('mean'), col('std'), lit(0.93)))\
                .withColumn('p_94', compute_delay_percentile(col('mean'), col('std'), lit(0.94)))\
                .withColumn('p_95', compute_delay_percentile(col('mean'), col('std'), lit(0.95)))\
                .withColumn('p_96', compute_delay_percentile(col('mean'), col('std'), lit(0.96)))\
                .withColumn('p_97', compute_delay_percentile(col('mean'), col('std'), lit(0.97)))\
                .withColumn('p_98', compute_delay_percentile(col('mean'), col('std'), lit(0.98)))\
                .withColumn('p_99', compute_delay_percentile(col('mean'), col('std'), lit(0.99)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Save to hdfs:

In [22]:
actual.write.format("orc").mode('overwrite').save("/user/{}/delay_distribution_percentiles.orc".format(username))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…